In [36]:
import sys
sys.path.append("..")

from turbochat.v1.prompt import GPTMsges, GPTToolPrompt
from turbochat.v1.gpt import GPT, Msg, Tool
from Apollo.Conversation.converse.prompts.user import PromptMaker

import pandas as pd
import json

GPT_API_KEY_PATH = r"C:\Users\nagen\Desktop\gpt\project_apollo\Apollo\openai_key"

In [3]:
# gpt api key
with open(GPT_API_KEY_PATH, 'r') as f:
    gpt = GPT(f.read())

### tool definition

In [19]:
DEFINTION = {
    "type": "function",
    "function": {
        "name": "extract_user_health_information_entry",
        "description": """
            This tool will run all the time to extract smallest of the user details in the specified format.
            from user response, extracts information related to user's mental/physical health, medical information, habits, everything related to the user's health. Multiple information entries can be extracted.
            All sorts of entries describing user's lifestyle, medication, state, symptoms is extracted.
            Do not extract information about user goals.
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "entries": {
                    "type": "array",
                    "description": "list of information entries",
                    "items": {
                        "type": "object",
                        "properties": {
                            
                            "health_parameter": {
                                "type": "string",
                                "description": """
                                    health parameter,
                                    extracted perameter that will determine user's state / probles / conditions
                                    Example: BMI, weight, height, heart rate, stress level, back pain, sleep schedule, etc,
                                """
                            },
                            
                            "health_parameter_category": {
                                "type": "string",
                                "description": """
                                    category of health parameter propertie
                                    Example: Diet, Report, Mental, Lifestyle, symptom, habit etc
                                """
                            },
                            
                            "health_parameter_value": {
                                "type": "string",
                                "description": """
                                    value of health parameter propertie
                                    Example: 127cm, 40Kg, High, Low, Irregular, balanced, bad breadth, severe etc.
                                """
                            },
                        
                        },
                        "required": [
                            "health_parameter",
                            "health_parameter_category",
                            "health_parameter_value",
                        ],
                    }
                }
            },
        },
        "required": ["entries"],
    },
}

TOOL_PROMPT = GPTToolPrompt(DEFINTION)

### tool callable

In [51]:
def get_callable_df_with_columns(columns):
    def _callable(**kwargs):
        data = kwargs[list(kwargs.keys())[0]]
        df = pd.DataFrame(data)
        df.columns = columns
        return df
    return _callable

TOOL_CALLABLE = get_callable_df_with_columns(["i_parameter_label", "parameter_type", "parameter_value"])

### user prompt

In [45]:
pm = PromptMaker({
    "user_prompt": {"label": "user prompt"},
    "services": {"label": "Available services"},
    "doctors": {"label": "Available doctors"},
    "history": {"label": "User related information"},
})


prompt = pm.get({
    "user_prompt": "I have sever depression"
})

print(prompt)

TOOL_MESSAGES = GPTMsges([{
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": prompt
        }
    ]
}])

user prompt:
I have sever depression


### Tool Call

In [55]:
tool = Tool(gpt, TOOL_MESSAGES, TOOL_PROMPT, TOOL_CALLABLE)

In [56]:
response, tool_prompt, results = tool.call()

In [59]:
status, stash = results
stash["extract_user_health_information_entry"][0]

,i_parameter_label,parameter_type,parameter_value
0,depression,Mental,severe
